### Accuracy of different algorithms

In [176]:
import numpy as np
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, SVR
%matplotlib inline

# import file
data = pd.read_csv('facies_vectors.csv')

# fill NaN with the mean value
data.fillna(data['PE'].mean(), inplace=True)

# create a raining wells set of data (without test well)
training_wells = data[data['Well Name'] != 'SHANKLE']

# create a test well set of data
test_well = data[data['Well Name'] == 'SHANKLE']

# unique training well names
well_names = list(training_wells['Well Name'].unique())

# create a list of features from training wells (without Facies)
X_train = np.array(training_wells.drop(['Facies', 'Formation', 'Well Name', 
                                        'Depth', 'NM_M', 'RELPOS'], 1))

# create a label from training wells (Facies)
y_train = np.array(training_wells['Facies'])

# create a list of features from test well (without Facies)
X_test = np.array(test_well.drop(['Facies', 'Formation', 'Well Name', 
                                  'Depth', 'NM_M', 'RELPOS'], 1))

# create a label from test well (Facies)
y_test = np.array(test_well['Facies'])

models = []
models.append(('Lin_Reg', LinearRegression()))
models.append(('Log_Reg', LogisticRegression()))
models.append(('DTC', DecisionTreeClassifier()))
models.append(('KNC', KNeighborsClassifier()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('GNB', GaussianNB()))
models.append(('BR', BayesianRidge()))
models.append(('EN', ElasticNet()))
names = []
results = []

for name, model in models:
    model.fit(X_train, y_train)
    names.append(name)
    results.append(model.score(X_test, y_test))

for i in range(len(names)):
    print("%s: %s" % (names[i], results[i]))

Lin_Reg: 0.491938720496
Log_Reg: 0.412026726058
DTC: 0.336302895323
KNC: 0.398663697105
LDA: 0.516703786192
GNB: 0.556792873051
BR: 0.491023182681
EN: 0.0968876481818


### Let's count the number of each facies in the test well SHANKLE:

In [206]:
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00',
       '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']

# 1=sandstone  2=c_siltstone   3=f_siltstone  # 4=marine_silt_shale 
# 5=mudstone 6=wackestone 7=dolomite 8=packstone 9=bafflestone
facies_labels = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS',
                 'WS', 'D','PS', 'BS']

facies_titles = ['sandstone', 'c_siltstone', 'f_siltstone', 'marine_silt_shale',
                'mudstone', 'wackestone', 'dolomite', 'packstone', 'bafflestone']

facies_color_map = dict(zip(facies_labels, facies_colors))

d = {}

for i in y_test:
    d[i] = d.get(i, 0) + 1
    
print(d)

{1: 89, 2: 89, 3: 117, 4: 7, 5: 19, 6: 71, 7: 17, 8: 40}


### Confusion matrix

In [213]:
from sklearn.metrics import confusion_matrix

prediction_GNB = GNB.predict(X_test)

print(confusion_matrix(y_test, prediction_GNB))

[[60 19  5  0  0  5  0  0  0]
 [24 51  9  0  0  5  0  0  0]
 [ 0 44 67  0  1  4  0  1  0]
 [ 0  0  0  1  0  6  0  0  0]
 [ 0  0  0  9  0 10  0  0  0]
 [ 0  0  1  5  4 50  0 10  1]
 [ 4  1  1  1  0  0  4  6  0]
 [ 0  0  1  6  0 16  0 17  0]
 [ 0  0  0  0  0  0  0  0  0]]


However, let's check confusion matrix with other algorithms:

In [208]:
prediction_Log_Reg = Log_Reg.predict(X_test)
prediction_DTC = DTC.predict(X_test)
prediction_KNC = KNC.predict(X_test)
prediction_LDA = LDA.predict(X_test)

print(confusion_matrix(y_test, prediction_Log_Reg))
print("-" * 30)
print(confusion_matrix(y_test, prediction_DTC))
print("-" * 30)
print(confusion_matrix(y_test, prediction_KNC))
print("-" * 30)
print(confusion_matrix(y_test, prediction_LDA))

[[ 0 85  1  0  0  3  0  0]
 [ 0 80  6  0  0  2  0  1]
 [ 0 51 57  0  0  4  0  5]
 [ 0  1  0  0  0  6  0  0]
 [ 0  2  0  6  0  8  0  3]
 [ 0 13  1  2  0 22  0 33]
 [ 0  9  7  0  0  0  0  1]
 [ 0  2  0  1  0 11  0 26]]
------------------------------
[[ 4 55 11  9  3  4  0  2  1]
 [17 31 20  7  2  7  1  4  0]
 [ 4 39 52  5  7  3  3  4  0]
 [ 0  0  0  0  0  6  0  1  0]
 [ 0  0  0 11  0  2  0  5  1]
 [ 0  2  2  3 10 40  1 13  0]
 [ 2  0  0  1  3  1  4  6  0]
 [ 0  1  0  1  3  7  1 23  4]
 [ 0  0  0  0  0  0  0  0  0]]
------------------------------
[[32 42  7  4  1  0  0  3  0]
 [13 47 20  4  0  3  0  1  1]
 [ 6 48 52  2  5  1  2  1  0]
 [ 0  5  0  0  0  2  0  0  0]
 [ 0  3  0  6  1  0  2  7  0]
 [ 1  5  2  2  7 31  3 18  2]
 [ 2  0  1  0  0  1  3  9  1]
 [ 0  1  0  2  2 12  6 13  4]
 [ 0  0  0  0  0  0  0  0  0]]
------------------------------
[[31 55  1  0  0  2  0  0  0]
 [ 1 76  6  0  0  5  0  1  0]
 [ 0 49 59  0  0  5  0  4  0]
 [ 0  0  1  0  0  5  0  1  0]
 [ 0  3  0  9  0  5  0  2  0

In [ ]:
# describe the main statistical distribution of the input variables
training_wells.describe()

training_wells.groupby('Facies').size()

training_wells.plot(kind='box', subplots=True, layout=(3, 3), figsize=(7, 7), 
                    fontsize=8, sharex=False, sharey=False)
plt.show()

training_wells.hist(figsize=(7, 7))
plt.show()

scatter_matrix(training_wells)
plt.show()

In [ ]:
# print out all the empty values
print(data.isnull().sum())

In [ ]:
# ALL TRAINING WELLS
fig = plt.figure(figsize=(20, 10)) 

# (1, 5, 1) stands for (from, to, where)
ax1 = fig.add_subplot(1, 5, 1)
ax1.scatter(data['GR'], data['Depth'], color='r', alpha=0.5)
ax1.set_ylabel('Depth', fontsize = '14' )
ax1.set_xlabel('GR', fontsize = '14')
plt.gca().invert_yaxis()

ax2 = fig.add_subplot(1, 5, 2)
ax2.scatter(data['ILD_log10'], data['Depth'], color = 'g', alpha=0.5)
ax2.set_xlabel('ILD_log10', fontsize = '14')
plt.gca().invert_yaxis()

ax3 = fig.add_subplot(1, 5, 3)
ax3.scatter(data['DeltaPHI'], data['Depth'], color = 'r', alpha=0.5)
ax3.set_xlabel('DeltaPHI', fontsize = '14')
plt.gca().invert_yaxis()

ax4 = fig.add_subplot(1, 5, 4)
ax4.scatter(data['PHIND'], data['Depth'], color = 'k', alpha=0.5)
ax4.set_xlabel('PHIND', fontsize = '14')
plt.gca().invert_yaxis()

ax4 = fig.add_subplot(1, 5, 5)
ax4.scatter(data['PE'], data['Depth'], color = 'k', alpha=0.5)
ax4.set_xlabel('PE', fontsize = '14')
plt.gca().invert_yaxis()


plt.show()

In [ ]:
# Lokking for GR extremums
num = 0
for i in training_wells['GR']:
    if i >= 150:
        print(i, facies_titles[data['Facies'][num]])
    num += 1
    
    

#num = 0
#for i in training_wells['GR']:
#    if i >= 150:
#        print('GR %d  -->  %d m  -->  %s' % (i, training_wells['Depth'][num], facies_titles[training_wells['Facies'][num]]))
#    num += 1

In [ ]:
# SHRIMPLIN WELL
SW = data[data['Well Name'] == 'SHRIMPLIN']

fig = plt.figure(figsize=(20, 10)) 

# (1, 5, 1) stands for (from, to, where)
ax1 = fig.add_subplot(1, 5, 1)
ax1.scatter(SW['GR'], SW['Depth'], color='r', alpha=0.5)
ax1.set_ylabel('Depth', fontsize = '14' )
ax1.set_xlabel('GR', fontsize = '14')
plt.gca().invert_yaxis()

ax2 = fig.add_subplot(1, 5, 2)
ax2.scatter(SW['ILD_log10'], SW['Depth'], color = 'g', alpha=0.5)
ax2.set_xlabel('ILD_log10', fontsize = '14')
plt.gca().invert_yaxis()

ax3 = fig.add_subplot(1, 5, 3)
ax3.scatter(SW['DeltaPHI'], SW['Depth'], color = 'r', alpha=0.5)
ax3.set_xlabel('DeltaPHI', fontsize = '14')
plt.gca().invert_yaxis()

ax4 = fig.add_subplot(1, 5, 4)
ax4.scatter(SW['PHIND'], SW['Depth'], color = 'k', alpha=0.5)
ax4.set_xlabel('PHIND', fontsize = '14')
plt.gca().invert_yaxis()

ax4 = fig.add_subplot(1, 5, 5)
ax4.scatter(SW['PE'], SW['Depth'], color = 'k', alpha=0.5)
ax4.set_xlabel('PE', fontsize = '14')
plt.gca().invert_yaxis()


plt.show()

In [ ]:
# SHRIMPLIN WELL: looking for the GR extremums, its Depth and Facies
num = 0
for i in SW['GR']:
    if i >= 150:
        print('GR %d --> %d m --> %s' % (i, float(SW['Depth'][num]), facies_titles[SW['Facies'][num]]))
    num += 1

#### Performed in cooperation with Ievgen Ustenko
Github: https://github.com/iUstenko |
LinkedIn: https://goo.gl/gfRoSc